# Combined model Notebook

Train a logistic regression model (one per each tear) to combine the output of each plane to obtain a final probability for each tear.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Import necessary libraries

In [ ]:
#import all libraries
import torch
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.utils.data as data
import numpy as np
import pandas as pd
# Set a random seed for reproducibility
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
torch.manual_seed(42)

In [ ]:
%%capture
!pip install xlwt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Model Definitions

In [ ]:
#add another fully connected layer to convert output (1,1000) to (1)
class BaselineNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model = models.resnet18(weights='ResNet18_Weights.DEFAULT')   # delete the last fc layer.
        self.classifer = nn.Linear(1000, 1)


    def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x) #output size (s, 1000)
        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 1000)
        output =nn.ReLU()(output)
        output = self.classifer(output) #output size (1)

        return output

In [ ]:
class axial_model(nn.Module):
  def __init__(self, dropout1, dropout2, bias1, bias2, activation1, activation2, stride1, stride2):
        super().__init__()
        self.pretrained_model = nn.Sequential(*list(models.resnet18(weights='ResNet18_Weights.DEFAULT').children())[:-2])
        self.conv1 = nn.Sequential(
            nn.Conv2d(512, 64, 4, stride=stride1,bias=bias1),
            nn.Dropout(p=dropout1),
            activation1,
            nn.BatchNorm2d(64),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 32, 4, stride=stride2,bias=bias2),
            nn.Dropout(p=dropout2),
            activation2,
            nn.BatchNorm2d(32),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifer = nn.Linear(32,1)

        for param in self.pretrained_model.parameters():
          param.requires_grad = False


  def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.avgpool(x)
        x = self.flatten(x)

        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 1000)
        output =nn.ReLU()(output)
        output = self.classifer(output) #output size (1)

        return output

### Load model and weights
Particular model/ weights required uncommented at time runtime.

In [ ]:
acl_sagittal_model = BaselineNet().to(device)
acl_coronal_model = BaselineNet().to(device)
acl_axial_model = axial_model(0.4061, 0.395, 1.0, 0.8248, nn.LeakyReLU(), nn.Tanh() , 1, 1).to(device) # parameters obtained from tuning notebook

meniscus_sagittal_model = BaselineNet().to(device)
meniscus_coronal_model = BaselineNet().to(device)
meniscus_axial_model = axial_model(dropout1= 0.2,  dropout2= 0.493, bias1= 0.4213, bias2= 1,activation1 = nn.Sigmoid(),activation2 = nn.Tanh(), stride1 = 1, stride2 = 1).to(device)

abnormal_sagittal_model = BaselineNet().to(device)
abnormal_coronal_model = BaselineNet().to(device)
abnormal_axial_model = axial_model(0.406, 0.4504, 0.6919, 0.3155, nn.LeakyReLU(), nn.Tanh(), 1, 1).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 155MB/s]


#### ACL

In [ ]:
acl_sagittal_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/Baseline_Model_ACL_weights_sagittal.pth'))
acl_coronal_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/Baseline_Model_ACL_weights_coronal.pth'))
acl_axial_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/HyperTrainResNet18Adapt_model_axial.pth'))

<All keys matched successfully>

#### Meniscus

In [ ]:
meniscus_sagittal_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/Baseline_Model_meniscus_weights_sagittal.pth'))
meniscus_coronal_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/Baseline_Model_meniscus_weights_coronal.pth'))
meniscus_axial_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/model_weights/HyperTrainResNet18Adapt_model_axial_meniscus.pth'),strict=False) # Emmet Edit
#meniscus_axial_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/HyperTrainResNet18Adapt_model_axial.pth'))

_IncompatibleKeys(missing_keys=['conv2.0.bias'], unexpected_keys=[])

#### Abnormal

In [ ]:
abnormal_sagittal_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/Baseline_Model_abnormal_weights_sagittal.pth'))
abnormal_coronal_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/Baseline_Model_abnormal_weights_coronal.pth'))
#abnormal_axial_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/HyperTrainResNet18Adapt_model_axial_abnormal.pth',strict=False))
abnormal_axial_model.load_state_dict(torch.load('/content/gdrive/Shareddrives/MRNet Project/final_model_weights/HyperTrainResNet18Adapt_model_axial.pth'))

<All keys matched successfully>

## Data Loading

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, split='train', transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.split=split
        if self.split == 'train':
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        elif self.split == 'test':
            self.folder_path = self.root_dir + 'test/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'test-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform

        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index]) #load MRI
        label = self.labels[index] #get label of MRI
        label = torch.FloatTensor([label]) #convert type from numpy to torch

        if self.transform: #if you are transforming it
            array = self.transform(array) #transform the image
            array = array.numpy()


        array = np.stack((array,)*3, axis=1) #the model expects dimensions of (3, 256, 256), the MRIs are greyscale of size (256, 256). Therefore, we stack the image three times to fit the dimensions for the model.
        array = torch.FloatTensor(array)

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        return array, label, weight

# Load Train Datasets

In [ ]:
def format_weights(acl='acl', meniscus='meniscus', abnormal='abnormal'):
  return {'acl':acl, 'meniscus':meniscus, 'abnormal':abnormal}

In [ ]:
directory='/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/' #directory to the data
tasks = format_weights()
planes = {'sagittal':'sagittal', 'axial':'axial', 'coronal':'coronal'}

# define a compose function that is a series of transformations on the images.
augmentor = transforms.Compose([
            # transforms.Lambda(lambda x: torch.Tensor(x)), #converts from numpy to tensor
            transforms.ToTensor(),  # Convert images to tensors
            transforms.Resize((299, 299), antialias=True),  # Resize images to 299by299
            transforms.RandomRotation(25), #rotate the image by 25 degrees
            transforms.RandomAffine([0.11, 0.11]), #blur the edges
            transforms.RandomHorizontalFlip() #flip the image
        ])

## initialise the train datasets (class we defined earlier) and then initialise a Pytorch's dataloader
# acl dataset + dataloader

acl_sagittal_train_dataset = Dataset(directory, tasks['acl'], planes['sagittal'], split='train', transform = augmentor)
acl_coronal_train_dataset = Dataset(directory, tasks['acl'], planes['coronal'], split='train', transform = augmentor)
acl_axial_train_dataset = Dataset(directory, tasks['acl'], planes['axial'], split='train', transform = augmentor)
acl_sagittal_train_loader = DataLoader(acl_sagittal_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
acl_coronal_train_loader = DataLoader(acl_coronal_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
acl_axial_train_loader = DataLoader(acl_axial_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

# acl dataset + dataloader
meniscus_sagittal_train_dataset = Dataset(directory, tasks['meniscus'], planes['sagittal'], split='train', transform = augmentor)
meniscus_coronal_train_dataset = Dataset(directory, tasks['meniscus'], planes['coronal'], split='train', transform = augmentor)
meniscus_axial_train_dataset = Dataset(directory, tasks['meniscus'], planes['axial'], split='train', transform = augmentor)
meniscus_sagittal_train_loader = DataLoader(meniscus_sagittal_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
meniscus_coronal_train_loader = DataLoader(meniscus_coronal_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
meniscus_axial_train_loader = DataLoader(meniscus_axial_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

# acl dataset + dataloader
abnormal_sagittal_train_dataset = Dataset(directory, tasks['abnormal'], planes['sagittal'], split='train', transform = augmentor)
abnormal_coronal_train_dataset = Dataset(directory, tasks['abnormal'], planes['coronal'], split='train', transform = augmentor)
abnormal_axial_train_dataset = Dataset(directory, tasks['abnormal'], planes['axial'], split='train', transform = augmentor)
abnormal_sagittal_train_loader = DataLoader(abnormal_sagittal_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
abnormal_coronal_train_loader = DataLoader(abnormal_coronal_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
abnormal_axial_train_loader = DataLoader(abnormal_axial_train_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)


# Load Test Datasets

In [ ]:
directory='/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/' #directory to the data
tasks = format_weights()
planes = {'sagittal':'sagittal', 'axial':'axial', 'coronal':'coronal'}

transf = transforms.Compose([
            transforms.ToTensor(),  # Convert images to tensors
            transforms.Resize((224, 224), antialias=True),  # Resize images to 299by299
        ])

## initialise the test datasets (class we defined earlier) and then initialise a Pytorch's dataloader
# acl dataset + dataloader
acl_sagittal_test_dataset = Dataset(directory, tasks['acl'], planes['sagittal'], split='test', transform = transf)
acl_coronal_test_dataset = Dataset(directory, tasks['acl'], planes['coronal'], split='test', transform = transf)
acl_axial_test_dataset = Dataset(directory, tasks['acl'], planes['axial'], split='test', transform = transf)
acl_sagittal_test_loader = DataLoader(acl_sagittal_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
acl_coronal_test_loader = DataLoader(acl_coronal_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
acl_axial_test_loader = DataLoader(acl_axial_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

# acl dataset + dataloader
meniscus_sagittal_test_dataset = Dataset(directory, tasks['meniscus'], planes['sagittal'], split='test', transform = transf)
meniscus_coronal_test_dataset = Dataset(directory, tasks['meniscus'], planes['coronal'], split='test', transform = transf)
meniscus_axial_test_dataset = Dataset(directory, tasks['meniscus'], planes['axial'], split='test', transform = transf)
meniscus_sagittal_test_loader = DataLoader(meniscus_sagittal_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
meniscus_coronal_test_loader = DataLoader(meniscus_coronal_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
meniscus_axial_test_loader = DataLoader(meniscus_axial_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

# acl dataset + dataloader
abnormal_sagittal_test_dataset = Dataset(directory, tasks['abnormal'], planes['sagittal'], split='test', transform = transf)
abnormal_coronal_test_dataset = Dataset(directory, tasks['abnormal'], planes['coronal'], split='test', transform = transf)
abnormal_axial_test_dataset = Dataset(directory, tasks['abnormal'], planes['axial'], split='test', transform = transf)
abnormal_sagittal_test_loader = DataLoader(abnormal_sagittal_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
abnormal_coronal_test_loader = DataLoader(abnormal_coronal_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)
abnormal_axial_test_loader = DataLoader(abnormal_axial_test_dataset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

## Get Predictions

'Test' function, to return probabilites for each model

In [ ]:
def test(model,test_loader):
    _ = model.eval()
    probabilities = []
    labels = []

    with torch.no_grad():
        for i, (image, label, weight) in enumerate(test_loader):
          if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
          label = label[0]
          prediction = model(image.float().squeeze(0))
          probas = torch.sigmoid(prediction).item()
          probabilities.append(probas)
          label_arr = (label.detach().cpu().numpy()).flatten()
          label_numeric = label_arr[0]
          labels.append(label_numeric)

          # raise Exception()

    return labels, probabilities

### Predictions

Get Test predictions

In [ ]:
acl_test_sagittal_labels, acl_test_sagittal_preds = test(acl_sagittal_model, acl_sagittal_test_loader)
acl_test_coronal_labels, acl_test_coronal_preds = test(acl_coronal_model, acl_coronal_test_loader)
acl_test_axial_labels, acl_test_axial_preds = test(acl_axial_model, acl_axial_test_loader)

meniscus_test_sagittal_labels, meniscus_test_sagittal_preds = test(meniscus_sagittal_model, meniscus_sagittal_test_loader)
meniscus_test_coronal_labels, meniscus_test_coronal_preds = test(meniscus_coronal_model, meniscus_coronal_test_loader)
meniscus_test_axial_labels, meniscus_test_axial_preds = test(meniscus_axial_model, meniscus_axial_test_loader)

abnormal_test_sagittal_labels, abnormal_test_sagittal_preds = test(abnormal_sagittal_model, abnormal_sagittal_test_loader)
abnormal_test_coronal_labels, abnormal_test_coronal_preds = test(abnormal_coronal_model, abnormal_coronal_test_loader)
abnormal_test_axial_labels, abnormal_test_axial_preds = test(abnormal_axial_model, abnormal_axial_test_loader)

In [ ]:
df_test_labels = pd.DataFrame(list(zip(acl_test_sagittal_labels, acl_test_coronal_labels, acl_test_axial_labels, meniscus_test_sagittal_labels, meniscus_test_coronal_labels, meniscus_test_axial_labels, abnormal_test_sagittal_labels, abnormal_test_coronal_labels, abnormal_test_axial_labels)),
               columns =['acl_sagittal_labels', 'acl_coronal_labels', 'acl_axial_labels', 'meniscus_sagittal_labels','meniscus_coronal_labels', 'meniscus_axial_labels', 'abnormal_sagittal_labels', 'abnormal_coronal_labels', 'abnormal_axial_labels'])
df_test_labels.head()

,acl_sagittal_labels,acl_coronal_labels,acl_axial_labels,meniscus_sagittal_labels,meniscus_coronal_labels,meniscus_axial_labels,abnormal_sagittal_labels,abnormal_coronal_labels,abnormal_axial_labels
0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
df_test_labels.to_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/test_labels.xlsx')

In [ ]:
df_test_preds = pd.DataFrame(list(zip(acl_test_sagittal_preds, acl_test_coronal_preds, acl_test_axial_preds, meniscus_test_sagittal_preds, meniscus_test_coronal_preds, meniscus_test_axial_preds, abnormal_test_sagittal_preds, abnormal_test_coronal_preds, abnormal_test_axial_preds)),
               columns =['acl_sagittal_preds', 'acl_coronal_preds', 'acl_axial_preds', 'meniscus_sagittal_preds','meniscus_coronal_preds', 'meniscus_axial_preds', 'abnormal_sagittal_preds', 'abnormal_coronal_preds', 'abnormal_axial_preds'])
df_test_preds.head()

,acl_sagittal_preds,acl_coronal_preds,acl_axial_preds,meniscus_sagittal_preds,meniscus_coronal_preds,meniscus_axial_preds,abnormal_sagittal_preds,abnormal_coronal_preds,abnormal_axial_preds
0,0.678587,0.733679,0.528554,0.688433,0.669842,0.471683,0.980243,0.560353,0.528554
1,0.002532,0.186442,0.528554,0.151005,0.069886,0.472287,0.400259,0.439465,0.528554
2,0.079797,0.024415,0.528554,0.631517,0.003899,0.471989,0.022398,0.009967,0.528554
3,0.688770,0.008239,0.528554,0.605831,0.338343,0.472177,0.944850,0.997886,0.528554
4,0.000125,0.022707,0.528554,0.163657,0.166278,0.472147,0.896589,0.879186,0.528554


In [ ]:
df_test_preds.to_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/test_raw_predictions.xlsx')

Get Train predictions

In [ ]:
acl_train_sagittal_labels, acl_train_sagittal_preds = test(acl_sagittal_model, acl_sagittal_train_loader)
acl_train_coronal_labels, acl_train_coronal_preds = test(acl_coronal_model, acl_coronal_train_loader)
acl_train_axial_labels, acl_train_axial_preds = test(acl_axial_model, acl_axial_train_loader)

meniscus_train_sagittal_labels, meniscus_train_sagittal_preds = test(meniscus_sagittal_model, meniscus_sagittal_train_loader)
meniscus_train_coronal_labels, meniscus_train_coronal_preds = test(meniscus_coronal_model, meniscus_coronal_train_loader)
meniscus_train_axial_labels, meniscus_train_axial_preds = test(meniscus_axial_model, meniscus_axial_train_loader)

abnormal_train_sagittal_labels, abnormal_train_sagittal_preds = test(abnormal_sagittal_model, abnormal_sagittal_train_loader)
abnormal_train_coronal_labels, abnormal_train_coronal_preds = test(abnormal_coronal_model, abnormal_coronal_train_loader)
abnormal_train_axial_labels, abnormal_train_axial_preds = test(abnormal_axial_model, abnormal_axial_train_loader)

In [ ]:
df_train_labels = pd.DataFrame(list(zip(acl_train_sagittal_labels, acl_train_coronal_labels, acl_train_axial_labels, meniscus_train_sagittal_labels, meniscus_train_coronal_labels, meniscus_train_axial_labels, abnormal_train_sagittal_labels, abnormal_train_coronal_labels, abnormal_train_axial_labels)),
               columns =['acl_train_sagittal_labels', 'acl_train_coronal_labels', 'acl_train_axial_labels', 'menisucs_train_sagittal_labels','menisucs_train_coronal_labels', 'menisucs_train_axial_labels', 'abonormal_train_sagittal_labels', 'abonormal_train_coronal_labels', 'abonormal_train_axial_labels'])
df_train_labels.head()

In [ ]:
df_train_labels.to_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/train_labels.xlsx')

In [ ]:
df_train_preds = pd.DataFrame(list(zip(acl_train_sagittal_preds, acl_train_coronal_preds, acl_train_axial_preds, meniscus_train_sagittal_preds, meniscus_train_coronal_preds, meniscus_train_axial_preds, abnormal_train_sagittal_preds, abnormal_train_coronal_preds, abnormal_train_axial_preds)),
               columns =['acl_train_sagittal_preds', 'acl_train_coronal_preds', 'acl_train_axial_preds', 'menisucs_train_sagittal_preds','menisucs_train_coronal_preds', 'menisucs_train_axial_preds', 'abonormal_train_sagittal_preds', 'abonormal_train_coronal_preds', 'abonormal_train_axial_preds'])
df_train_preds.head()

In [ ]:
df_train_preds.to_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/train_raw_predictions.xlsx')

# Logistic Regression model to combine the output

Train three logistic regression models (one per each tear) to combine the ouput of each plane to obtain a final probability for each task(tear)

Read predictions and labels of the training dataset

In [ ]:
#df_train_labels = pd.read_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/train_labels.xlsx')
df_train_labels = pd.read_excel('/content/gdrive/MyDrive/assignment/train_labels.xlsx',index_col=0)
df_train_labels.head()

,acl_train_sagittal_labels,acl_train_coronal_labels,acl_train_axial_labels,menisucs_train_sagittal_labels,menisucs_train_coronal_labels,menisucs_train_axial_labels,abonormal_train_sagittal_labels,abonormal_train_coronal_labels,abonormal_train_axial_labels
0,0,0,0,0,0,0,1,1,1
1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,1,1,1
3,0,0,0,1,1,1,1,1,1
4,0,0,0,0,0,0,1,1,1


In [ ]:
#df_train_preds = pd.read_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/train_raw_predictions.xlsx')
df_train_preds = pd.read_excel('/content/gdrive/MyDrive/assignment/train_raw_predictions.xlsx',index_col=0)
df_train_preds.head()

,acl_train_sagittal_preds,acl_train_coronal_preds,acl_train_axial_preds,menisucs_train_sagittal_preds,menisucs_train_coronal_preds,menisucs_train_axial_preds,abonormal_train_sagittal_preds,abonormal_train_coronal_preds,abonormal_train_axial_preds
0,0.438566,0.721292,0.509197,0.466317,0.511121,0.459192,0.974108,0.788288,0.509200
1,0.847582,0.935666,0.509242,0.781748,0.797569,0.458934,0.927309,0.639197,0.509260
2,0.495543,0.794163,0.509159,0.496072,0.215310,0.458863,0.159602,0.706634,0.509177
3,0.768576,0.555872,0.509111,0.388101,0.767925,0.459160,0.656275,0.210503,0.509109
4,0.548514,0.637607,0.509146,0.072635,0.183140,0.458591,0.465382,0.608017,0.509149


Read predictions and labels of the test dataset

In [ ]:
#df_test_labels = pd.read_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/test_labels.xlsx')
df_test_labels = pd.read_excel('/content/gdrive/MyDrive/assignment/test_labels.xlsx',index_col=0)
df_test_labels.head()

,acl_sagittal_labels,acl_coronal_labels,acl_axial_labels,meniscus_sagittal_labels,meniscus_coronal_labels,meniscus_axial_labels,abnormal_sagittal_labels,abnormal_coronal_labels,abnormal_axial_labels
0,0,0,0,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,1,1
3,0,0,0,0,0,0,1,1,1
4,0,0,0,1,1,1,1,1,1


In [ ]:
#df_test_preds = pd.read_excel('/content/gdrive/Shareddrives/MRNet Project/predictions/test_raw_predictions.xlsx')
df_test_preds = pd.read_excel('/content/gdrive/MyDrive/assignment/test_raw_predictions.xlsx',index_col=0)
df_test_preds.head()

,acl_sagittal_preds,acl_coronal_preds,acl_axial_preds,meniscus_sagittal_preds,meniscus_coronal_preds,meniscus_axial_preds,abnormal_sagittal_preds,abnormal_coronal_preds,abnormal_axial_preds
0,0.678587,0.733679,0.528554,0.688433,0.669842,0.471683,0.980243,0.560353,0.528554
1,0.002532,0.186442,0.528554,0.151005,0.069886,0.472287,0.400259,0.439465,0.528554
2,0.079797,0.024415,0.528554,0.631517,0.003899,0.471989,0.022398,0.009967,0.528554
3,0.688770,0.008239,0.528554,0.605831,0.338343,0.472177,0.944850,0.997886,0.528554
4,0.000125,0.022707,0.528554,0.163657,0.166278,0.472147,0.896589,0.879186,0.528554


In [ ]:
#TO BE DELETED for testing the implementation in small data
#df_train_labels = df_train_labels.head(5)
#df_train_preds = df_train_preds.head(5)

Pre-process training data

In [ ]:
yall_train = []

In [ ]:
yall_train.extend(df_train_labels['abonormal_train_axial_labels'].tolist())
yall_train.extend(df_train_labels['abonormal_train_coronal_labels'].tolist())
yall_train.extend(df_train_labels['abonormal_train_sagittal_labels'].tolist())
yall_train.extend(df_train_labels['acl_train_axial_labels'].tolist())
yall_train.extend(df_train_labels['acl_train_coronal_labels'].tolist())
yall_train.extend(df_train_labels['acl_train_sagittal_labels'].tolist())
yall_train.extend(df_train_labels['menisucs_train_axial_labels'].tolist())
yall_train.extend(df_train_labels['menisucs_train_coronal_labels'].tolist())
yall_train.extend(df_train_labels['menisucs_train_sagittal_labels'].tolist())

In [ ]:
len(yall_train)

9090

In [ ]:
Xall_train = []  # Abnormal, ACL, Meniscus

In [ ]:
X = [df_train_preds['abonormal_train_axial_preds'], df_train_preds['abonormal_train_coronal_preds'], df_train_preds['abonormal_train_sagittal_preds']]
Xall_train.append(X)
X = [df_train_preds['acl_train_axial_preds'], df_train_preds['acl_train_coronal_preds'], df_train_preds['acl_train_sagittal_preds']]
Xall_train.append(X)
X = [df_train_preds['menisucs_train_axial_preds'], df_train_preds['menisucs_train_coronal_preds'], df_train_preds['menisucs_train_sagittal_preds']]
Xall_train.append(X)

In [ ]:
yall_train = np.asarray(yall_train).transpose()
Xall_train = np.asarray(Xall_train)

In [ ]:
Xall_train.shape

(3, 3, 1010)

In [ ]:
Xall_train[0].shape

(3, 1010)

In [ ]:
dim_of_tear_train = Xall_train[0].shape[0]*Xall_train[0].shape[1]
dim_of_tear_train

3030

Pre-process of test dataset

In [ ]:
yall_test = []

In [ ]:
df_test_labels.columns

Index(['acl_sagittal_labels', 'acl_coronal_labels', 'acl_axial_labels',
       'meniscus_sagittal_labels', 'meniscus_coronal_labels',
       'meniscus_axial_labels', 'abnormal_sagittal_labels',
       'abnormal_coronal_labels', 'abnormal_axial_labels'],
      dtype='object')

In [ ]:
yall_test.extend(df_test_labels['abnormal_axial_labels'].tolist())
yall_test.extend(df_test_labels['abnormal_coronal_labels'].tolist())
yall_test.extend(df_test_labels['abnormal_sagittal_labels'].tolist())
yall_test.extend(df_test_labels['acl_axial_labels'].tolist())
yall_test.extend(df_test_labels['acl_coronal_labels'].tolist())
yall_test.extend(df_test_labels['acl_sagittal_labels'].tolist())
yall_test.extend(df_test_labels['meniscus_axial_labels'].tolist())
yall_test.extend(df_test_labels['meniscus_coronal_labels'].tolist())
yall_test.extend(df_test_labels['meniscus_sagittal_labels'].tolist())

In [ ]:
len(yall_test)

1080

In [ ]:
Xall_test = []  # Abnormal, ACL, Meniscus

In [ ]:
X = [df_test_preds['abnormal_axial_preds'], df_test_preds['abnormal_coronal_preds'], df_test_preds['abnormal_sagittal_preds']]
Xall_test.append(X)
X = [df_test_preds['acl_axial_preds'], df_test_preds['acl_coronal_preds'], df_test_preds['acl_sagittal_preds']]
Xall_test.append(X)
X = [df_test_preds['meniscus_axial_preds'], df_test_preds['meniscus_coronal_preds'], df_test_preds['meniscus_sagittal_preds']]
Xall_test.append(X)

In [ ]:
yall_test = np.asarray(yall_test).transpose()
Xall_test = np.asarray(Xall_test)

In [ ]:
Xall_test.shape

(3, 3, 120)

In [ ]:
Xall_test[0].shape

(3, 120)

In [ ]:
dim_of_tear_test = Xall_test[0].shape[0]*Xall_test[0].shape[1]
dim_of_tear_test

360

Logistic Regression

In [ ]:
tasks = ['abnormal', 'acl', 'meniscus']

In [ ]:
for i in range(3):
    # get part of training that coresponds to a specific task
    X = Xall_train[i]
    X = X.flatten().reshape(-1,1)
    if(i==0):
      y = yall_train[0:dim_of_tear_train]
      y_test = yall_test[0:dim_of_tear_test]
      #y=yall_train[0:360]
      #y=v[0:15]
    elif(i==1):
      y = yall_train[dim_of_tear_train:dim_of_tear_train*2]
      y_test = yall_test[dim_of_tear_test:dim_of_tear_test*2]
      #y =yall_train[360:720]
      #y=yall_train[15:30]
    else:
      y = yall_train[dim_of_tear_train*2:dim_of_tear_train*3]
      y_test = yall_test[dim_of_tear_test*2:dim_of_tear_test*3]
      #y = yall_train[720:1080]
      #y=yall_train[30:45]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    #y_preds_valid = logreg.predict(X_valid)
    print('------- Task: {} -------'.format(tasks[i]))
    print("---- Training & Validation -----")
    print('Accuracy of logistic regression classifier on validation set: {:.2f}'.format(logreg.score(X_valid, y_valid)))
    y_probas_valid = logreg.predict_proba(X_valid)[:,1]
    print('Validation - Probability for: {} is {:.2f}\n'.format(tasks[i], sum(y_probas_valid) / len(y_probas_valid)))

    X_test = Xall_test[i]
    X_test = X_test.flatten().reshape(-1,1)

    y_probas_test = logreg.predict_proba(X_test)[:,1]
    print("---- Evaluation -----")
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    print('Evaluation - Probability for: {} is {:.2f}\n'.format(tasks[i], sum(y_probas_test) / len(y_probas_test)))

------- Task: abnormal -------
---- Training & Validation -----
Accuracy of logistic regression classifier on validation set: 0.83
Validation - Probability for: abnormal is 0.82

---- Evaluation -----
Accuracy of logistic regression classifier on test set: 0.81
Evaluation - Probability for: abnormal is 0.81

------- Task: acl -------
---- Training & Validation -----
Accuracy of logistic regression classifier on validation set: 0.84
Validation - Probability for: acl is 0.19

---- Evaluation -----
Accuracy of logistic regression classifier on test set: 0.85
Evaluation - Probability for: acl is 0.12

------- Task: meniscus -------
---- Training & Validation -----
Accuracy of logistic regression classifier on validation set: 0.67
Validation - Probability for: meniscus is 0.34

---- Evaluation -----
Accuracy of logistic regression classifier on test set: 0.65
Evaluation - Probability for: meniscus is 0.31



Cross Validation

In [ ]:
print(f'Training logistic regression models for each task/tear...')

clfs = []

for i in range(3):
    X = Xall_train[i]
    if(i==0):
      y=yall_train[0:dim_of_tear_train]
      #y=yall_train[0:360]
      #y=v[0:15]
    elif(i==1):
      y=yall_train[dim_of_tear_train:dim_of_tear_train*2]
      #y =yall_train[360:720]
      #y=yall_train[15:30]
    else:
      y=yall_train[dim_of_tear_train*2:dim_of_tear_train*3]
      #y = yall_train[720:1080]
      #y=yall_train[30:45]
    print('X ',X.shape)
    print('y ',len(y))
    X = X.flatten().reshape(-1,1)
    clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, y)
    print(f'Cross validation score for {tasks[i]}: {clf.score(X, y):.3f}')

Training logistic regression models for each task/tear...
X  (3, 1010)
y  3030
Cross validation score for abnormal: 0.826
X  (3, 1010)
y  3030
Cross validation score for acl: 0.839
X  (3, 1010)
y  3030
Cross validation score for meniscus: 0.698
